In [5]:
import tkinter as tk
import win32gui
from PIL import ImageGrab
import keras
import numpy as np

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.x = self.y = 0
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognize", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        
        self.canvas.grid(row=0, column=0, pady=2, sticky=tk.W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
        
        self.model = keras.models.load_model('mnist_optimized.h5')
    
    def clear_all(self):
        self.canvas.delete("all")
    
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()
        rect = win32gui.GetWindowRect(HWND)
        a, b, c, d = rect
        rect = (a + 4, b + 4, c - 4, d - 4)
        im = ImageGrab.grab(rect).convert('L')
        
        im = im.resize((28, 28))
        im = np.array(im)
        im = im.reshape(1, 28, 28, 1)
        
        digit, acc = self.predict_digit(im)
        
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')
        
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')
    
    def predict_digit(self, image):
        prediction = self.model.predict(image)
        digit = np.argmax(prediction)
        acc = prediction[0][digit]
        return digit, acc

if __name__ == "__main__":
    app = App()
    app.mainloop()


1/1 [==============================] - 0s 18ms/step
